## Grid World Enviroment setting
* states, actions, transition probability

In [2]:
# set state
import numpy as np
nCols = 3
nRows = 4
nWalls = 1
states = []
for i in range(nCols*nRows-nWalls):
    states.append(i)
N_STATES = len(states)

terminal_state = [3,6]
win_state = [3]
lose_state = [6]
start_state = [x for x in states if x not in terminal_state]
#print(N_STATES)
#print(states)

# set map
map = -np.ones((nCols+2,nRows+2))
for i in range(nCols):
    for j in range(nRows):
        map[i+1,j+1] = 0
map[2,2] = -1 # add wall
#print(map)

# set action
actions = [0, 1, 2, 3]
N_ACTIONS = len(actions)

# states -> location
locations = []
index = 0
for i in range(nCols):
    for j in range(nRows):
        if map[i+1,j+1]==0:
            locations.append((i+1,j+1))
            index = index + 1
#print(locations) # match index with states
# action -> move
move = [(0,-1),(-1,0),(0,1),(1,0)] # match index with actions
#print(move)

# set transition probability
P = np.zeros((N_STATES,N_ACTIONS,N_STATES)) # P[S,A,S']
for s in range(N_STATES):
    for a in range(N_ACTIONS):
        current_location = locations[s]
        # heading collectly  ####################################################################################
        next_location = (current_location[0] + move[a][0],current_location[1] + move[a][1])
        
        if map[next_location[0],next_location[1]] == -1: # there is barrier or wall
            next_location = current_location
            next_s = states[locations.index(next_location)]
        else:
            next_s = states[locations.index(next_location)]
        P[s,a,next_s] = P[s,a,next_s] + 0.8
        # left error ############################################################################################
        next_location = (current_location[0] + move[a-1][0],current_location[1] + move[a-1][1])
        if map[next_location[0],next_location[1]] == -1: # there is barrier or wall
            next_location = current_location
            next_s = states[locations.index(next_location)]
        else:
            next_s = states[locations.index(next_location)]
        P[s,a,next_s] = P[s,a,next_s] + 0.1
        # right error ############################################################################################
        next_location = (current_location[0] + move[(a+1)%4][0],current_location[1] + move[(a+1)%4][1])
        
        if map[next_location[0],next_location[1]] == -1: # there is barrier or wall
            next_location = current_location
            next_s = states[locations.index(next_location)]
        else:
            next_s = states[locations.index(next_location)]
        P[s,a,next_s] = P[s,a,next_s] + 0.1
        
# rewards s,a ---  R(s,a)  ---> s'
if True:
    R = -0.02*np.ones((N_STATES,N_ACTIONS))
else:
    R = -0.5*np.ones((N_STATES,N_ACTIONS))
R[3,:] = 1
R[6,:] = -1
#print(R)
# discount factor
gamma = 0.99

# policy : given state which action would u choose
# assume that we know the policy
bad_policy = np.zeros((N_STATES,N_ACTIONS))
bad_policy[0,2] = 1
bad_policy[1,2] = 1
bad_policy[2,2] = 1
bad_policy[3,2] = 1
bad_policy[4,3] = 1
bad_policy[5,2] = 1
bad_policy[6,2] = 1
bad_policy[7,2] = 1
bad_policy[8,2] = 1
bad_policy[9,2] = 1
bad_policy[10,1] = 1

random_policy = 0.25*np.ones((N_STATES,N_ACTIONS))

optimal_policy = np.zeros((N_STATES,N_ACTIONS))
optimal_policy[0,2] = 1
optimal_policy[1,2] = 1
optimal_policy[2,2] = 1
optimal_policy[3,2] = 1
optimal_policy[4,1] = 1
optimal_policy[5,1] = 1
optimal_policy[6,1] = 1
optimal_policy[7,1] = 1
optimal_policy[8,0] = 1
optimal_policy[9,0] = 1
optimal_policy[10,0] = 1
#print(optimal_policy)

optimalWithNoise_policy = np.zeros((N_STATES,N_ACTIONS))
ep = 0.1
optimalWithNoise_policy[0,2] = 1
optimalWithNoise_policy[1,2] = 1
optimalWithNoise_policy[2,2] = 1
optimalWithNoise_policy[3,2] = 1
optimalWithNoise_policy[4,1] = 1
optimalWithNoise_policy[5,1] = 1
optimalWithNoise_policy[6,1] = 1
optimalWithNoise_policy[7,1] = 1
optimalWithNoise_policy[8,0] = 1
optimalWithNoise_policy[9,0] = 1
optimalWithNoise_policy[10,0] = 1
optimalWithNoise_policy = optimalWithNoise_policy + (ep/4)*np.ones((N_STATES,N_ACTIONS))
optimalWithNoise_policy = optimalWithNoise_policy / np.sum(optimalWithNoise_policy,axis = 1).reshape((N_STATES,1))

In [3]:
import tensorflow as tf

/home/donghyun/anaconda2/envs/RL_STUDY/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/donghyun/anaconda2/envs/RL_STUDY/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/donghyun/anaconda2/envs/RL_STUDY/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/donghyun/anaconda2/envs/RL_S

## Every visit Monte Carlo Policy Evaluation
* Function Approximation(linear combination)
$$V(s) = X(s)^{T}w$$
* loss function
$$J(w) = E_{\pi}[(G_{t}\:-\:X(s)^{T}w)^{2}]$$
* gradient descent
$$\nabla_{w}J(w) = -2\:*\:E_{\pi}[(G_{t}\:-\:X(s)^{T}w)]\:*\:X(s)$$
* stochastic gradient descent(batch size 1)
$$\nabla_{w}J(w) = -2\:*(G_{t}\:-\:X(s)^{T}w)\:*\:X(s)$$
* update parameter vector w
$$\Delta w = \alpha\:*\:(G_{t}\:-\:X(s)^{T}w)*X(s) $$

In [4]:
# Every-visit Monte Carlro Policy Evaluation for V

tf.reset_default_graph()
import time
start = time.time()
print("Tensorflow version : ")
print(tf.__version__)
print()


## set HyperParemeters
epoch = 1000
lr_rate = 0.01
policy = optimalWithNoise_policy # Evaluation -> follow given policy
## MC evaluation
num_visit = np.zeros(N_STATES) # N(s)
cum_gain = np.zeros(N_STATES) # S(s)
with tf.device('/gpu:0'):
    ## set tensorflow variable
    state_tf = tf.placeholder(tf.int32,shape=[None],name = "state")
    gain_tf = tf.placeholder(tf.float32,shape=[None],name = 'gain')
    #### number state -> matrix ex. 3 -> [0 0 0 1 0 0 0 0 0 0 0]
    W = tf.get_variable(name='W', \
                        shape = [N_STATES,1],\
                        dtype = tf.float32, \
                        initializer=tf.random_uniform_initializer(-1.0,1.0))
state_tf_one_hot = tf.one_hot(state_tf,N_STATES)
V = tf.matmul(state_tf_one_hot, W) # linear combination reprentaion of state value function
MC_error = gain_tf - V
loss = tf.reduce_mean(tf.square(MC_error)) #mean-square-error
opt = tf.train.GradientDescentOptimizer(learning_rate=lr_rate)
train_ops = opt.minimize(loss)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for _ in range(epoch):
        
        #print(str(_+1)+"th iteration")
        done = False
        reward_history = []
        simulation_history = []
        gain_history = []   
        s = np.random.choice(start_state) # random initial state
        
        while not done:
            simulation_history.append(s)
            a = np.random.choice(actions,p=policy[s,:])
            reward_history.append(R[s,a])
            s1 = np.random.choice(states,p=P[s,a,:])
            
            if s1 in terminal_state:
                done = True
                simulation_history.append(s1)
                reward_history.append(R[s1,0])
            
            else:
                s = s1
                
        # After finish one simulation update value function -> offline
        # evaluate G(t)
        for i,r in enumerate(reward_history[::-1]):
            # G(t-1) = reward(t) + gamma * G(t)
            # if terminal G(T) = r(T)
            # To implent, i use reverse ordering
            if i==0:
                gain_history.append(r)
            else:
                gain_history.append(gamma * gain_history[i-1] + r)

        gain_history = gain_history[::-1]
        ##-------------------- This is for Exact MC
        # add G(t) to s(t)
        for i,s in enumerate(simulation_history):
            # i for find G(t)
            # S(s) = S(s) + G(t) for only first visit.
            num_visit[s]+=1
            cum_gain[s]= cum_gain[s] + gain_history[i]
        ##-------------------- This is for Function approximation MC
        for i in range(len(simulation_history)):
            feed_dict = {state_tf: [simulation_history[i]],\
                        gain_tf: [gain_history[i]]}
            sess.run(train_ops, feed_dict=feed_dict)
    # after finish all epoch
    V_final=[]
    for s in states:
        feed_dict = {state_tf: [s]}
        V_now = sess.run(V,feed_dict=feed_dict)
        V_final.append(V_now[0][0])
    print("Function Approximation result")
    print(V_final)
V = np.zeros(N_STATES)
V = cum_gain/(num_visit+1.0e-8)
print()
print()
print("Exact Value Function from MC")
print(V)
print()
print()
print("it takes "+str(round(time.time()-start))+" sec")

Tensorflow version : 
1.12.0

Function Approximation result
[0.87889445, 0.92578095, 0.95151484, 0.9999985, 0.8379802, 0.6346723, -0.7479279, 0.7984123, 0.75648016, 0.69450295, 0.2999555]


Exact Value Function from MC
[ 0.84628378  0.89640149  0.92440533  1.          0.80435099  0.62801648
 -1.          0.75364188  0.7062999   0.63806861  0.24453983]


it takes 5 sec


In [5]:
# Every-visit Monte Carlro Policy Evaluation for Q

tf.reset_default_graph()
import time
start = time.time()
print("Tensorflow version : ")
print(tf.__version__)
print()


## set HyperParemeters
epoch = 1000
lr_rate = 0.01
policy = optimalWithNoise_policy # Evaluation -> follow given policy

with tf.device('/cpu:0'):
    ## set tensorflow variable
    state_tf = tf.placeholder(tf.int32,shape=[None],name = "state")
    action_tf = tf.placeholder(tf.int32,shape=[None],name = "action")
    gain_tf = tf.placeholder(tf.float32,shape=[None],name = 'gain')
    #### number state -> matrix ex. 3 -> [0 0 0 1 0 0 0 0 0 0 0]
    W = tf.get_variable(name='W', \
                        shape = [N_STATES+N_ACTIONS,1],\
                        dtype = tf.float32, \
                        initializer=tf.random_uniform_initializer(-1.0,1.0))
state_tf_one_hot = tf.one_hot(state_tf,N_STATES)
action_tf_one_hot = tf.one_hot(action_tf,N_ACTIONS)

state_action_tf_one_hot = tf.concat([state_tf_one_hot, action_tf_one_hot], 1)

Q = tf.matmul(state_action_tf_one_hot, W) # linear combination reprentaion of state value function
MC_error = gain_tf - Q
loss = tf.reduce_mean(tf.square(MC_error)) #mean-square-error
opt = tf.train.GradientDescentOptimizer(learning_rate=lr_rate)
train_ops = opt.minimize(loss)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    
    for _ in range(epoch):
        reward_history = []
        simulation_history = []
        gain_history = []   
        #print(str(_+1)+"th iteration")
        done = False
        s = np.random.choice(start_state) # random initial state
        
        while not done:
            simulation_history.append((s,a))
            a = np.random.choice(actions,p=policy[s,:])
            reward_history.append(R[s,a])
            s1 = np.random.choice(states,p=P[s,a,:])
            
            if s1 in terminal_state:
                done = True
                simulation_history.append((s1,0))
                reward_history.append(R[s1,0])
            
            else:
                s = s1
                
        # After finish one simulation update value function -> offline
        # evaluate G(t)
        for i,r in enumerate(reward_history[::-1]):
            # G(t-1) = reward(t) + gamma * G(t)
            # if terminal G(T) = r(T)
            # To implent, i use reverse ordering
            if i==0:
                gain_history.append(r)
            else:
                gain_history.append(gamma * gain_history[i-1] + r)

        gain_history = gain_history[::-1]

        ##-------------------- This is for Function approximation MC
        for i,(s,a) in enumerate(simulation_history):
            feed_dict = {state_tf: [s],\
                         action_tf: [a],\
                        gain_tf: [gain_history[i]]}
            sess.run(train_ops, feed_dict=feed_dict)
            
    # after finish all epoch
    Q_final= np.empty((N_STATES,N_ACTIONS))
    for s in states:
        for a in actions:
            feed_dict = {state_tf: [s],action_tf: [a]}
            Q_now = sess.run(Q,feed_dict=feed_dict)
            Q_final[s,a] = Q_now[0][0]
    print("Function Approximation result")
    print(Q_final)

print("it takes "+str(round(time.time()-start))+" sec")

Tensorflow version : 
1.12.0

Function Approximation result
[[ 0.83177966  0.79616362  0.83210868  0.89934504]
 [ 0.88527918  0.84966314  0.8856082   0.9528445 ]
 [ 0.9282003   0.89258426  0.92852932  0.99576569]
 [ 1.02505088  0.98943484  1.0253799   1.09261632]
 [ 0.77365315  0.73803711  0.77398217  0.84121853]
 [ 0.65365565  0.61803961  0.65398467  0.72122103]
 [-0.5377652  -0.57338125 -0.53743619 -0.47019982]
 [ 0.72090089  0.68528485  0.72122991  0.78846622]
 [ 0.67953688  0.64392084  0.6798659   0.74710226]
 [ 0.67015076  0.63453472  0.67047977  0.73771614]
 [ 0.46425289  0.42863685  0.46458191  0.53181827]]
it takes 7 sec
